In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  5 11:53:29 2022

@author: Raghav Agarwal
"""
import requests
import json
import os
import datetime
from typing import List, Dict
import pandas as pd
import pytz
import requests
import sys,os
import time
import socket
import pandas as pd
import numpy as np
import pytz
#from influxdb import InfluxDBClient, DataFrameClient

# # thrones clients
# from thrones.database.InfluxClient import InfluxClient
# from thrones.database.PostgreClient import PostgreClient

# # constants and models
# # import thrones.database.database_common.consts as db_consts
# from thrones.database.database_common.models.spot_orderbook_tick import get_spot_orderbook_tick_model
# import thrones.database.database_common.consts as db_consts
# import thrones.analytics.common.consts as consts
# import thrones.analytics.utils as AnalyticsUtil
# import thrones.exchange.common.CheckUtil as CheckUtil
# from thrones.util.TimeUtil import (datetime_to_timestamp, string_to_standard_datetime, timestamp_to_hkt_datetime, timestamp_to_gmt_datetime)
# import thrones.exchange.gateio.gateio_common.gateio_sdk.consts as gateio_consts

# # SQL
# from sqlalchemy import text, create_engine
# from sqlalchemy.ext.declarative import declarative_base


# from thrones.util.TimeUtil import (
#     datetime_to_millisec,
#     string_to_millisec,
#     string_to_standard_datetime,
#     timestamp_to_hkt_datetime, 
#     timestamp_to_gmt_datetime
# )
class historical_orderbook:

    def __init__(self):
        # if os.name.lower() == 'nt':
        #     self.influx_host_name = db_consts.TDB_INFLUX_HOST_LOCAL
        #     self.influx_port = db_consts.TDB_INFLUX_PORT_LOCAL
        #     self.host_name = db_consts.PDB_POSTGRE_HOST_LOCAL
        #     self.port = db_consts.PDB_POSTGRE_PORT_LOCAL
        #     self.db_name = consts.DB_NAME
        # else:
        #     if socket.gethostname() == 'HX-HK-WS-U019':  #testing ubuntu machine
        #         self.influx_host_name = db_consts.TDB_INFLUX_HOST_LOCAL
        #         self.influx_port = db_consts.TDB_INFLUX_PORT_LOCAL
        #         self.postgre_host_name = db_consts.PDB_POSTGRE_HOST_LOCAL
        #         self.postgre_port = db_consts.PDB_POSTGRE_PORT_LOCAL
        #     else:
        #         self.influx_host_name = db_consts.TDB_INFLUX_HOST_PROD
        #         self.influx_port = db_consts.TDB_INFLUX_PORT_PROD
        #         self.postgre_host_name = db_consts.PDB_POSTGRE_HOST_PROD
        #         self.postgre_port = db_consts.PDB_POSTGRE_PORT_PROD
        pass

    def daterange(start_date, end_date):
            for n in range(int((end_date - start_date).days)):
                yield start_date + datetime.datetime.timedelta(n)

    def orderbook(self, symbol:str, start_time , end_time) -> None:
        # postgre_client = PostgreClient(host=self.host_name,
        #                        port=self.port,
        #                        user=os.getenv("PDB_POSTGRE_HTM_USER"),
        #                        pwd=os.getenv("PDB_POSTGRE_HTM_PWD"),
        #                        db_name=self.db_name)
        # CheckUtil.check_start_end_time(start_time)
        # if isinstance(start_time, datetime.datetime): 
        #     start_time = datetime_to_timestamp(date_time=start_time)
        # else: 
        #     start_time = datetime_to_timestamp(string_to_standard_datetime(date_str=start_time))
        # CheckUtil.check_start_end_time(end_time)
        # if isinstance(end_time, datetime.datetime):  
        #     end_time = datetime_to_timestamp(date_time=end_time)
        # else:
        #     end_time = datetime_to_timestamp(string_to_standard_datetime(date_str=end_time))

        start_timestamp = start_time
        end_timestamp = end_time
        # sql_str = text('''select min(timestamp) from \"spot_orderbook_tick_gt\" where symbol = '{}' '''.format(symbol))
        # d = pd.read_sql(sql_str, con=postgre_client.engine)
        # if d.empty:
        #     t = 1000000000000000
        # else:
        #     t = d.at[0,"min"]
        t = 1000000000000000
        def daterange(start_date, end_date):
            for n in range(int((end_date - start_date).days)):
                yield start_date + datetime.timedelta(n)
        # sql_str1 = text(''' select * from \"ref_market\" where symbol = '{}' and exchange = 'GT' '''.format(symbol))
        # m_id = pd.read_sql(sql_str1, con=postgre_client.engine)
        # market_id = m_id.at[0,"market_id"]
        market_id = 156
        column__ = ['timestamp', 'datetime_HKT', 'datetime_GMT', 'market_id', 'market', 'symbol'] + [j+"_"+str(i)+"_"+z for j in ["ask","bid"] for z in ["px","qty"] for i in range(0,20)]
        column__temp = ['timestamp','market_id', 'market', 'symbol'] + [j+"_"+str(i)+"_"+z for j in ["ask","bid"] for z in ["px","qty"] for i in range(0,20)]
        start_date = datetime.datetime.utcfromtimestamp(start_time).date() 
        end_date = datetime.datetime.utcfromtimestamp(end_time).date() +datetime.timedelta(days=1)
        print(start_date)
        top20 = pd.DataFrame(columns = column__)
        for single_date in daterange(start_date, end_date):
            top20 = pd.DataFrame(columns = column__)
            for i in range(0,24):
                try:
                    time1 = datetime.datetime.combine(single_date, datetime.time(hour = i))
                    print(str(time1.timestamp())+"  "+str(start_timestamp))
                    if(time1.timestamp()+9*60*60>=start_timestamp):
                        url = "https://download.gatedata.org"+"/spot/orderbooks/"+ single_date.strftime("%Y%m")+"/"+symbol.replace("-","_")+"-"+single_date.strftime("%Y%m%d")+"{:02d}".format(i)+".csv.gz"
                        print(url)
                        filename = url.split("/")[-1]
                        with open(filename, "wb") as f:
                            r = requests.get(url)
                            f.write(r.content)
                        df = pd.read_csv(filename,  compression='gzip')
                        df.columns = ["timestamp", "side", "action","price","amount","ignore1","ignore2"]
                        df.loc[df["side"]==1, "side"] = "sell"
                        df.loc[df["side"]==2, "side"] = "buy"
                        df = df.drop(columns={"ignore1","ignore2"})

                        init_ord_book = df.loc[df["action"]=="set"]
                        remaining_ord_book = df.loc[df["action"]!="set"]
                        temp = remaining_ord_book.groupby(['timestamp'])
                        df_final = pd.DataFrame(columns = ['timestamp', 'side', 'action', 'price', 'amount'])  
                        for group_name, df_group in temp:
                            for index,row in df_group.iterrows():
                                if(row["action"]=="take"):
                                    init_ord_book.loc[(init_ord_book["price"] == row["price"]) & (init_ord_book["side"]==row["side"]), "amount"] -= row["amount"]
                                    init_ord_book = init_ord_book[init_ord_book.amount != 0]
                                elif(row["action"]=="make"):
                                    if(row["price"] in list(init_ord_book.loc[(row["side"] == init_ord_book["side"]), "price"]) ):
                                        init_ord_book.loc[(init_ord_book["price"]==row["price"]) & (init_ord_book["side"]==row["side"]), "amount"] += row["amount"]
                                    else:
                                        init_ord_book.loc[-1] = row
                                        init_ord_book.index = init_ord_book.index + 1
                            init_ord_book = init_ord_book[init_ord_book["amount"] != 0]
                            init_ord_book = init_ord_book[(init_ord_book["amount"] > 0.000000001)]
                            init_ord_book["timestamp"] = group_name
                            init_ord_book = init_ord_book.sort_values(by=['price'])
                            init_ord_book = init_ord_book.reset_index(drop=True)
                            temp1 = init_ord_book.loc[(init_ord_book['side']=="buy"),["price","amount"]].sort_values(by = ["price"], ascending = False)
                            bid_px = list(temp1["price"][0:20])
                            bid_qty = list(temp1["amount"][0:20])
                            temp2 = init_ord_book.loc[(init_ord_book['side']=="sell"),["price","amount"]].sort_values(by = ["price"], ascending = True)
                            ask_px = list(temp2["price"][0:20])
                            ask_qty = list(temp2["amount"][0:20])
                            final_row = [group_name*1000, market_id, "GT-"+symbol+"-SPOT", symbol]+ask_px + ask_qty + bid_px + bid_qty
                            temp_db = pd.DataFrame([final_row], columns = column__temp )
                            temp_db['datetime_HKT']=pd.to_datetime(temp_db['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Asia/Hong_Kong')

                            temp_db['datetime_GMT']=pd.to_datetime(temp_db['timestamp'], unit='ms')\
                                            .dt.tz_localize('UTC' )\
                                            .dt.tz_convert('Etc/GMT')
                            temp_db['datetime_HKT'] = pd.to_datetime(temp_db.datetime_HKT).dt.tz_localize(None)
                            temp_db['datetime_GMT'] = pd.to_datetime(temp_db.datetime_GMT).dt.tz_localize(None)
                            top20 = top20.append(temp_db, ignore_index=True)
                except:
                    pass
            try:

                top20.to_csv(symbol+"_orderbook_"+str(single_date)+".csv")
                print(single_date)
            except Exception as e:
                print(e)
        
        return top20
if __name__ == '__main__':
    # sample input historical_orderbook().orderbook(symbol = "", start_date= "", end_date = "")
    # https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083116.csv.gz
    df1 = historical_orderbook().orderbook(symbol = 'NEER-USDT', start_time =  1661904001, end_time = 1670817600)
    print(df1)

2022-08-31
1661904000.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083100.csv.gz


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


1661907600.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083101.csv.gz


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


1661911200.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083102.csv.gz
1661914800.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083103.csv.gz
1661918400.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083104.csv.gz


<ipython-input-1-b662f7448e85>:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  init_ord_book["timestamp"] = group_name


1661922000.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083105.csv.gz
1661925600.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083106.csv.gz
1661929200.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083107.csv.gz
1661932800.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083108.csv.gz
1661936400.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083109.csv.gz
1661940000.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083110.csv.gz
1661943600.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083111.csv.gz
1661947200.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083112.csv.gz
1661950800.0  1661904001
https://download.gatedata.org/spot/orderbooks/202208/NEER_USDT-2022083113.csv.gz
1661954400.0  1661904001
https://download.gate